In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("./Resources/cardio_train.csv", sep = ";", index_col=0)
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


<h3>1. Preprocessing</h3>

In [3]:
# convert naming of gluc
df.loc[df["gluc"] == 1, "gluc"] = "Glucose Normal"
df.loc[df["gluc"] == 2, "gluc"] = "Glucose Above Normal"
df.loc[df["gluc"] == 3, "gluc"] = "Glucose Well Above Normal"

# convert naming of cholesterol
df.loc[df["cholesterol"] == 1, "cholesterol"] = "Cholesterol Normal"
df.loc[df["cholesterol"] == 2, "cholesterol"] = "Cholesterol Above Normal"
df.loc[df["cholesterol"] == 3, "cholesterol"] = "Cholesterol Well Above Normal"

In [4]:
# Convert categorical n > 2 variable to one hot encoding.
categorical_dummies = pd.get_dummies(df[["cholesterol", "gluc"]], dtype=int)
categorical_dummies.columns = categorical_dummies.columns.str.replace("cholesterol_", "")
categorical_dummies.columns = categorical_dummies.columns.str.replace("gluc_", "")

df = pd.concat([df, categorical_dummies], axis=1)
df.drop(columns=["cholesterol", "gluc"], inplace=True)

df.head()


,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,Cholesterol Above Normal,Cholesterol Normal,Cholesterol Well Above Normal,Glucose Above Normal,Glucose Normal,Glucose Well Above Normal
id,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,0,0,1,0,0,1,0,0,1,0
1,20228,1,156,85.0,140,90,0,0,1,1,0,0,1,0,1,0
2,18857,1,165,64.0,130,70,0,0,0,1,0,0,1,0,1,0
3,17623,2,169,82.0,150,100,0,0,1,1,0,1,0,0,1,0
4,17474,1,156,56.0,100,60,0,0,0,0,0,1,0,0,1,0


In [5]:
#reorder the columns
df = df[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'smoke', 'alco',
       'active', 'Cholesterol Above Normal', 'Cholesterol Normal',
       'Cholesterol Well Above Normal', 'Glucose Above Normal',
       'Glucose Normal', 'Glucose Well Above Normal', 'cardio']]

df.head()

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,Cholesterol Above Normal,Cholesterol Normal,Cholesterol Well Above Normal,Glucose Above Normal,Glucose Normal,Glucose Well Above Normal,cardio
id,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,0,0,1,0,1,0,0,1,0,0
1,20228,1,156,85.0,140,90,0,0,1,0,0,1,0,1,0,1
2,18857,1,165,64.0,130,70,0,0,0,0,0,1,0,1,0,1
3,17623,2,169,82.0,150,100,0,0,1,0,1,0,0,1,0,1
4,17474,1,156,56.0,100,60,0,0,0,0,1,0,0,1,0,0


In [6]:
# split the data with train test split
X = df[df.columns[:-1]]
y = df["cardio"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Standardize the data
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

<h3>2. ML Models Initial Attempt</h3>

<h4>a. Logistic Regression with All Features</h4>

In [9]:
# try logistic regression model 
# define the model
clf = LogisticRegression(solver="lbfgs")

# fit the model
clf.fit(X_train_scaled, y_train)
logistic_predictions = clf.predict(X_test_scaled)

# calculate the accuracy score
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
logistic_accuracy


0.7188571428571429

<h4> b. Decision Tree with All Features </h4>

In [21]:
# create a decision feree classifier
from sklearn import tree
tree_clf = tree.DecisionTreeClassifier()

# fit the data and make prediction
tree_clf.fit(X_train_scaled, y_train)
dt_predictions = tree_clf.predict(X_test_scaled)

print(f"Decision Tree Accuracy is {accuracy_score(y_test, dt_predictions)}")

Accuracy is 0.6376


In [28]:
# see feature importance
print(tree_clf.feature_importances_)

[0.31960886 0.019423   0.14855789 0.15550587 0.22910484 0.04146374
 0.01002084 0.00770648 0.01476329 0.00622075 0.00624786 0.02074208
 0.00676121 0.00811575 0.00575752]


<h4> c. Random Forest with All Features </h4>

In [23]:
# create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200)

# fit the data to the model
rf_clf.fit(X_train_scaled, y_train)

# compute the accuracy
rf_predictions = rf_clf.predict(X_test_scaled)

print(f"Random Forest Accuracy is {accuracy_score(y_test, rf_predictions)}")


Random Forest Accuracy is 0.7171428571428572


In [29]:
# see feature importance
print(rf_clf.feature_importances_)

[0.30960041 0.0123561  0.1594796  0.17844434 0.16875189 0.09561478
 0.00743059 0.00645313 0.01199273 0.00470723 0.01581625 0.01647041
 0.00360716 0.0052956  0.00397978]


<h4> d. Feed Forward Network with All Features</h4>

In [30]:
# MLP Model
# model definition
mlp = tf.keras.models.Sequential()

# adding basic layers
mlp.add(tf.keras.layers.Dense(units = 6, activation = "relu", input_dim = 15))
mlp.add(tf.keras.layers.Dense(units = 6, activation = "relu"))
mlp.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
mlp.compile(loss = tf.keras.losses.BinaryCrossentropy, optimizer = optimizer, metrics = ["accuracy"])

# fit the mode
mlp.fit(X_train_scaled, y_train, epochs = 30)


Epoch 1/30


c:\Users\David\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 662us/step - accuracy: 0.6810 - loss: 0.6027
Epoch 2/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 684us/step - accuracy: 0.7305 - loss: 0.5485
Epoch 3/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 648us/step - accuracy: 0.7286 - loss: 0.5529
Epoch 4/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.7306 - loss: 0.5481
Epoch 5/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 631us/step - accuracy: 0.7302 - loss: 0.5480
Epoch 6/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 632us/step - accuracy: 0.7296 - loss: 0.5484
Epoch 7/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step - accuracy: 0.7286 - loss: 0.5503
Epoch 8/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 641us/step - accuracy: 0.7292 - loss: 0.5489
Epoch 9/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 652us/step - accuracy: 0.7316 - loss: 0.5465
Epoch 10/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 636us/step - accuracy: 0.7357 - loss: 0.5419
Epoch 11/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - accuracy: 0.7316 - loss: 0.5465
Epoch 12/30
1641/16

In [25]:
# evaluate the model, very bad, overfitted.
print(f"Accuracy: {mlp.evaluate(X_test, y_test)}")

547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - accuracy: 0.4997 - loss: 391.0298
Accuracy: [390.4146423339844, 0.5004000067710876]


<h3>3. ML Models With More Relevant Features</h3>

In [32]:
# split the data with train test split
X = df.iloc[:, [0, 2, 3, 4]]
y = df["cardio"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Standardize the data
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

<h4>a. Logistic Regression</h4>

In [34]:
logistic_clf = LogisticRegression(solver="lbfgs")

logistic_clf.fit(X_train_scaled, y_train)
logistic_predictions = logistic_clf.predict(X_test_scaled)

print(f"The accuracy of this logistic regression is {accuracy_score(y_test, logistic_predictions)}")

The accuracy of this logistic regression is 0.7082857142857143


<h4> b. Decision Tree </h4>

In [35]:
dt_clf = tree.DecisionTreeClassifier()
dt_clf.fit(X_train_scaled, y_train)
dt_predictions = dt_clf.predict(X_test_scaled)

print(f"The accuracy of this decision tree is {accuracy_score(y_test, dt_predictions)}")

The accuracy of this decision tree is 0.6201714285714286


<h3>c. Random Forest Classifier</h3>

In [36]:
# create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200)

# fit the data to the model
rf_clf.fit(X_train_scaled, y_train)

# compute the accuracy
rf_predictions = rf_clf.predict(X_test_scaled)

print(f"Random Forest Accuracy is {accuracy_score(y_test, rf_predictions)}")

Random Forest Accuracy is 0.6869714285714286


<h4> d. Feed Forward Network </h4>

In [38]:
# MLP Model
# model definition
mlp = tf.keras.models.Sequential()

# adding basic layers
mlp.add(tf.keras.layers.Dense(units = 6, activation = "relu", input_dim = 4))
mlp.add(tf.keras.layers.Dense(units = 6, activation = "relu"))
mlp.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
mlp.compile(loss = tf.keras.losses.BinaryCrossentropy, optimizer = optimizer, metrics = ["accuracy"])

# fit the mode
mlp.fit(X_train_scaled, y_train, epochs = 30)

Epoch 1/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 714us/step - accuracy: 0.6784 - loss: 0.6123
Epoch 2/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 716us/step - accuracy: 0.7157 - loss: 0.5658
Epoch 3/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 745us/step - accuracy: 0.7192 - loss: 0.5607
Epoch 4/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 744us/step - accuracy: 0.7190 - loss: 0.5581
Epoch 5/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - accuracy: 0.7180 - loss: 0.5580
Epoch 6/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 762us/step - accuracy: 0.7257 - loss: 0.5553
Epoch 7/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 733us/step - accuracy: 0.7154 - loss: 0.5589
Epoch 8/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 758us/step - accuracy: 0.7173 - loss: 0.5586
Epoch 9/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 724us/step - accuracy: 0.7189 - loss: 0.5579
Epoch 10/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 671us/step - accuracy: 0.7193 - loss: 0.5604
Epoch 11/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 705us/step - accuracy: 0.7238 - loss: 0.5571
Epoch 12